# Experiments on WinoLogic: Using Pre-trained Objectives and Sequence Classification
This notebook contains the implementation of baseline 1 and baseline 3 for WinoLogic

### Neural Language Models (NLMs)
The specific NLMs used:
- GPT-2: small and large ('gpt2', 'gpt2-large')
- BERT: base and large ('bert-base-uncased', 'bert-large-cased')
- RoBERTa: base and large ('roberta-base, 'roberta-large')
- BERT+WSCR: small and large ('bert-base-uncased', 'bert-large-uncased', both are fine-tuned on the WSCR dataset)
- RoBERTa+WSCR: small and large ('roberta-base', 'roberta-large-uncased', both are fine-tuned on the WSCR dataset)
- BERT+WinoGrande: small and large ('bert-base-uncased', 'bert-large-uncased', both are fine-tuned on the WinoGrande dataset)
- RoBERTa+WinoGrande: small and large ('roberta-base', 'roberta-large-uncased', both are fine-tuned on the WinoGrande dataset)
- BERT+QNLI/MNLI: small ('bert-base-uncased', is fine-tuned on the QNLI and MNLI datasets)
- RoBERTa+QNLI: small ('roberta-base', is fine-tuned on the QNLI dataset) 
- RoBERTa+MNLI: small and large ('roberta-base', 'roberta-large', both are fine-tuned on the MNLI dataset) 

The first three NLMs and RoBERTa+MNLI(large) are downloaded from [HuggingFace](https://huggingface.co/models), while the middle four NLMs could be obtained from [Kocijan et al.](https://github.com/vid-koci/bert-commonsense). The last three NLMs except RoBERTa+MNLI(large) are downloaded from [HuggingFace/textattack](https://huggingface.co/textattack)

### WinoLogic
WinoLogic is a dataset of 562 sentences, deriving from the [Winogrand Schema Challenge](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html). Each sentence is a concatenation of three parts:
- < WSC sentence >: The corresponding WSC sentence;
- < Answer >: The correct answer;
- < Knowledge sentence >: A plausibly positive knowledge sentence that could be used to reason the answer.

Given such a sentence, systems are required to decide whether the knowledge sentence could be used to reason the correct answer, and to label it accordingly.

##### Data Structure of WinoLogic and WinoWhy

An instance of WinoLoigc problem contains the following fields:
- `sentence`: The whole WinoLogic sentence
- `wsc_sentence`: The corresponding WSC sentence
- `answer_knowledge`: The correct answer and the plausible knowledge sentence
- `knowledge`: The plausible knowledge sentence
- `context`: The partial WinoLogic sentence until the unknown pronoun (inclusive)
- `label`: 0 or 1, 1 if and only if the knowledge sentence is positive
- `wsc_id`: The corresponding WSC problem ID, ranging in 0 to 272
- `fold_num`: The fold number ranging from 0 to 4
- `kID`: unique knowledge sentence ID
- `valid`: 0 or 1, 1 if and only if the knowledge sentence is valid
- `false_type`: 'negated', 'replaced', 'switched', 'changed', 'other'. Denotes that how the negative knowledge generate from the positive knowledge

The concatenation of `wsc_sentence` and `answer_knowledge` is a whole WinoLogic sentence, `sentence`.

The WinoWhy sentence has a similar structure, and thus has the same fields, except for
- `answer_reason`: The correct answer and the plausible reason
- `answer`: The plausible reason

There are three versions of winologic, named 'variable', 'grounded', 'natural'. All datasets are stored in the `datasets` folder, using the [JSON Lines](https://jsonlines.org/) format:
- `./datasets/winologic_variable.jsonl`: variable WinoLogic;
- `./datasets/winologic_grounded.jsonl`: grounded WinoLogic;
- `./datasets/winologic_natural.jsonl`: natural WinoLogic;
- `./datasets/winowhy.jsonl`: WinoWhy dataset.

### Implementation for baseline 1

For baseline 1, we use the pre-trained objectives to reduce each WinoLoigc sentence into cross-entropy values, then we predict whether a sentence is positive (label it with 1) or negative (label it with 0).

For each WinoWhy / WinoLogic sentence , it has the form [WSC sentence] [Answer] because
[Explanation or Knowledge sentence]. The [Answer] has the format "The [Pronoun] is more likely
to refer to [Candidate1] than [Candidate2]". Here the [Candidate1] is the correct candidate for the
pronoun.

Construct two text for that differ only on the position of [Candidate1] and [Candidate2]:
- s_1 is the same as : [WSC sentence] The [Pronoun] is more likely to refer to [Candidate1]
than [Candidate2] because [Explanation or Knowledge sentence]
- s_2 switches the two candidates: [WSC sentence] The [Pronoun] is more likely to refer to
[Candidate2] than [Candidate1] because [Explanation or Knowledge sentence]

If we can compare the probabilities of these two sentences, the model can make prediction
unsupervisedly. That is,
- If label = 1 and P(s_1) > P(s_2) then the model returns true
- If label = 0 and P(s_1) < P(s_2) then the model returns true
- Otherwise, false 
    
##### BERT/RoBERTa
For BERT/RoBERTa, we use the `BertForMaskedLM`/`RobertaForMaskedLM` to predict the masked tokens. Notice the pattern in `answer_knowledge` of WinoLogic (or `answer_reason` of WinoWhy): "... the < pronoun > is more likely to refer to < candidate1 > than < candidate2 > because ...". 
If a candidate has n tokens, we replace the candidate with n [MASK] symbols.

##### GPT-2
For GPT-2, we use the `GPT2LMHeadModel` from `transformers` as this model is pre-trained with the language modeling objective. For a WinoLogic sentence s = < txt1 > < pronoun > < txt2 > < answer_knowledge >. GPT-2 is given the whole sentence, but < txt1 > < pronoun > are masked. GPT-2 then use LM heads to compute the cross entropy loss for < txt2 > < answer_knowledge >.

### Implementation for baseline 3

For BERT/RoBERTa, we use the `BertForSequenceClassification`/`RobertaForSequenceClassification` to predict the label of winologic sentence. The models are given the input "[CLS] < wsc_sentence > [SEQ] < answer_knowledge > [SEQ]" and predict whether is "entailment" or "not entailment". The former corresponds to the label "1", and the latter corresponds to the label "0".

### Using NLMs

To use the GPT-2 models, change the `model_path` in ExpConfig instance, the directory should contain
- `pytorch_model.bin`: The LM model
- `vocab.json`: The vocabulary file
- `merges.txt`: The merges file
- `config.json`: The configuration of the model

To use the BERT models, change the `model_path` in ExpConfig instance, the directory should contain
- `pytorch_model.bin`: The LM model
- `vocab.txt`: The vocabulary file
- `config.json`: The configuration of the model

To use the RoBERTa models, change the `model_path` in ExpConfig instance, the directory should contain
- `pytorch_model.bin`: The LM model
- `vocab.json`: The vocabulary file
- `merges.txt`: The merges file
- `config.json`: The configuration of the model

Refer to both [HuggingFace](https://github.com/huggingface/transformers) and [Kocijan et al.](https://github.com/vid-koci/bert-commonsense) for more details.

To run the cells below, user should first assign the path values to the ROOT_PATH, WINOLOGIC_PATH, CACHE_DIR in the first cell.

In [1]:
import torch
import re
import numpy as np
import random
import json
import csv

from transformers import RobertaTokenizer, RobertaForMaskedLM, RobertaForSequenceClassification
from transformers import BertTokenizer, BertForMaskedLM, BertForSequenceClassification
from torch.nn import CrossEntropyLoss
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

ROOT_PATH = ''
WINOLOGIC_PATH = 'datasets/winologic_variable.jsonl'
CACHE_DIR = ''

In [2]:
# helper function: read and dump data
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [3]:
from dataclasses import dataclass

@dataclass
class WinoWhySentence(object):
    sentence: str = None
    context: str = None
    wsc_sentence: str = None
    answer_reason: str = None
    reason: str = None
    label: int = 0
    wsc_id: int = 0
    fold_num: int = 1
        
        
@dataclass
class WinoLogicSentence(object):
    kID: int = 0
    sentence: str = None
    context: str = None
    wsc_sentence: str = None
    wsc_marked_sentence: str = None
    answer_knowledge: str = None
    knowledge: str = None
    label: int = 0
    wsc_id: int = 0
    fold_num: int = 1
    valid: int = 1
    false_type: str = None
    
def load_winowhy_from_path(filepath: str):
    ws = load_jsonl(filepath)
    winowhy_sentences = list()
    for w in ws:
        s = WinoWhySentence()
        s.sentence = w['sentence']
        s.context = w['context']
        s.wsc_sentence = w['wsc_sentence']
        s.answer_reason = w['answer_reason']
        s.reason = w['reason']
        s.label = w['label']
        s.wsc_id = w['wsc_id']
        s.fold_num = w['fold_num']
        winowhy_sentences.append(s)
    return winowhy_sentences
        
def load_winologic_from_path(filepath: str):
    ws = load_jsonl(filepath)
    winologic_sentences = list()
    for w in ws:
        s = WinoLogicSentence()
        s.kID = w['kID']
        s.sentence = w['sentence']
        s.context = w['context']
        s.wsc_sentence = w['wsc_sentence']
        s.wsc_marked_sentence = w['wsc_marked_sentence']
        s.answer_knowledge = w['answer_knowledge']
        s.knowledge = w['knowledge']
        s.label = w['label']
        s.wsc_id = w['wsc_id']
        s.fold_num = w['fold_num'] - 1
        s.valid = w['valid']
        s.false_type = w['false_type']
        winologic_sentences.append(s)
    return winologic_sentences

@dataclass
class ExpConfig(object):
    # JSONL file path
    dataset_path: str = ""
    # "winologic", "winowhy"
    dataset: str = "winologic"
    # Task description
    task_name: str = ""
    # Only using single GPU
    gpu_id: int = 0
    # Seed for random
    seed: int = 42
    # 'cpu', 'cuda'
    device: str = 'cpu' 
    # "gpt2", "gpt2-large", "bert-base-uncased", "bert-large-uncased", "roberta-base", "roberta-largbe"
    model_name: str = ""
    # If model_path is not None or not empty, load model from model_path instead of transformers' pretrained ones
    model_path: str = ""
    # For training the classifier layer
    learning_rate: float = 1e-3
    # Number of total epochs
    num_training_epochs: int = 15
    # Max sequence length
    max_seq_len: int = 128

    def set_seed(self, new_seed = None):
        seed = self.seed if new_seed is None else new_seed
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    def set_gpu_if_possible(self, gpu_id = None):
        if torch.cuda.is_available():
            self.device = 'cuda'
            if gpu_id is not None:
                self.device = 'cuda:{}'.format(gpu_id)
        else:
            self.device = 'cpu'

## Use GPT-2

In [ ]:
def gpt2_cross_entropy_for_winologic_s1(wl_sentence: WinoLogicSentence, model: GPT2LMHeadModel, tokenizer: GPT2Tokenizer, device: str='cpu') -> float:
    """
    Returns the cross-entropy for a WinoLogic sentence. Given a WinoLogic sentence, use GPT-2 to predict the whole sentence,
    but we only report the the cross-entropy loss for the partial sentence after the pronoun in the WSC sentence.
    """
    
    full = wl_sentence.sentence
    
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')
    
    # Create the tokens for the WinoLogic sentence with cand1 replaced by masks [tokens for left] [<mask>]*len(tokens for cand1) [tokens for right]
    left_tokens = tokenizer.tokenize(full[:idx_cands_start], add_prefix_space=True)
    cand1_tokens = tokenizer.tokenize(cand1, add_prefix_space=True)
    cand2_tokens = tokenizer.tokenize(cand2, add_prefix_space=True)
    mid_tokens = tokenizer.tokenize('than', add_prefix_space=True)
    right_tokens = tokenizer.tokenize(full[idx_cands_end:], add_prefix_space=True)

    input_ids = tokenizer.convert_tokens_to_ids(left_tokens + cand1_tokens + mid_tokens + cand2_tokens + right_tokens)    
    
    context = wl_sentence.context
    full_input_ids = torch.tensor([input_ids]).to(device)
    #full_input_ids = torch.tensor(tokenizer.encode(full, add_prefix_space=True)).unsqueeze(0).to(device)
    ctx_input_ids = torch.tensor(tokenizer.encode(context, add_prefix_space=True)).unsqueeze(0).to(device)
    # let k be the length of context sequence
    k = ctx_input_ids.shape[-1]
    # Given a WinoLogic sentence of form <txt1> <pronoun> <txt2> <answer_knowledge>
    # only the prediction of the part2, <txt2> <answer_knowledge>, is used for calculating the loss
    # Create the correct labels: [-100 for txt1 and pronoun] [input ids for txt2 and answer_knowledge]
    # that is, [-100] * k + full_input_ids[:, k:]
    labels = torch.tensor([-100] * k).unsqueeze(0).to(device)
    labels = torch.cat((labels, full_input_ids[:, k:]), 1) 
    
    with torch.no_grad():
        part2_ce_loss, _, _ = model(full_input_ids, labels=labels)
    #print(part2_ce_loss.loss)
    return part2_ce_loss.to('cpu').item()

def gpt2_cross_entropy_for_winologic_s2(wl_sentence: WinoLogicSentence, model: GPT2LMHeadModel, tokenizer: GPT2Tokenizer, device: str='cpu') -> float:
    """
    Returns the cross-entropy for a WinoLogic sentence. Given a WinoLogic sentence, use GPT-2 to predict the whole sentence,
    but we only report the the cross-entropy loss for the partial sentence after the pronoun in the WSC sentence.
    """  
    
    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')
    
    # Create the tokens for the WinoLogic sentence with cand1 replaced by masks [tokens for left] [<mask>]*len(tokens for cand1) [tokens for right]
    left_tokens = tokenizer.tokenize(full[:idx_cands_start], add_prefix_space=True)
    cand1_tokens = tokenizer.tokenize(cand1, add_prefix_space=True)
    cand2_tokens = tokenizer.tokenize(cand2, add_prefix_space=True)
    mid_tokens = tokenizer.tokenize('than', add_prefix_space=True)
    right_tokens = tokenizer.tokenize(full[idx_cands_end:], add_prefix_space=True)

    input_ids = tokenizer.convert_tokens_to_ids(left_tokens + cand2_tokens + mid_tokens + cand1_tokens + right_tokens)
 
    context = wl_sentence.context
    
    full_input_ids = torch.tensor([input_ids]).to(device)
    #full_input_ids = torch.tensor(tokenizer.encode(full, add_special_tokens=True)).unsqueeze(0).to(device)
    ctx_input_ids = torch.tensor(tokenizer.encode(context, add_prefix_space=True)).unsqueeze(0).to(device)
    # let k be the length of context sequence
    k = ctx_input_ids.shape[-1]
    
    # Given a WinoLogic sentence of form <txt1> <pronoun> <txt2> <answer_knowledge>
    # only the prediction of the part2, <txt2> <answer_knowledge>, is used for calculating the loss
    # Create the correct labels: [-100 for txt1 and pronoun] [input ids for txt2 and answer_knowledge]
    # that is, [-100] * k + full_input_ids[:, k:]
    labels = torch.tensor([-100] * k).unsqueeze(0).to(device)
    labels = torch.cat((labels, full_input_ids[:, k:]), 1) 
    
    with torch.no_grad():
        part2_ce_loss, _, _ = model(full_input_ids, labels=labels)
    #print(part2_ce_loss.loss)
    return part2_ce_loss.to('cpu').item()


def gpt2_for_winologic(config: ExpConfig):
    """
    Using the GPT-2 model (`model_name`) to solve WinoLogic.
    Returns the list of accuracies.
    """
    # Load the WinoLogic dataset
    winologic_sentences = load_winologic_from_path(config.dataset_path)

    # set up model and tokenizer according to the config
    model_name = config.model_name
    # Load model from model_path if specified
    if config.model_path is not None and config.model_path != "":
        model_name = config.model_path
        
    # Init the model, tokenizer   
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.eval()
    model.to(config.device)
    
    # calculuate the cross-entropy loss for each WinoLogic sentence
    #result = open(SAVE_RESULT_PATH, "a")
    #writer = csv.writer(result)
    #writer.writerow(["kID","correct", "valid", "sentence", "wsc_id", "false_type", "label", "predict"])
    # calculuate the cross-entropy loss for each WinoLogic sentence
    correct = 0
    for ws in winologic_sentences:
        s1_loss = gpt2_cross_entropy_for_winologic_s1(ws, model, tokenizer, config.device)
        s2_loss = gpt2_cross_entropy_for_winologic_s2(ws, model, tokenizer, config.device)
        if (ws.label == 1 and s1_loss < s2_loss) or (ws.label == 0 and s1_loss > s2_loss):
            correct += 1
            #writer.writerow([ws.kID, 1, ws.valid, ws.sentence, ws.wsc_id, ws.false_type, ws.label, ws.label])
        #else:
            #writer.writerow([ws.kID, 0, ws.valid, ws.sentence, ws.wsc_id, ws.false_type, ws.label, 1 - ws.label])
            
    acc = correct / len(winologic_sentences)
    
    return acc

In [ ]:
for model_name in ['gpt2', 'gpt2-large']:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    gpt2config = ExpConfig()
    gpt2config.set_seed()
    gpt2config.set_gpu_if_possible(0)
    gpt2config.dataset = 'winologic'
    gpt2config.dataset_path = ROOT_PATH + WINOLOGIC_PATH
    gpt2config.task_name = 'Test on WinoLogic'
    gpt2config.model_name = model_name
    gpt2config.model_path = CACHE_DIR + model_name
    
    print('\n================================')
    print('Experiment: {} using {}'.format(gpt2config.task_name, gpt2config.model_name))
    
    scores = gpt2_for_winologic(gpt2config)
    
    print('Scores: {}'.format(scores))
    print('================================')

## Use BERT

In [ ]:
def bert_cross_entropy_for_winologic_s1(wl_sentence: WinoLogicSentence, model: BertForMaskedLM, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len
    
    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')
    
    # Create the tokens for the WinoLogic sentence with cand1 replaced by masks [tokens for left] [<mask>]*len(tokens for cand1) [tokens for right]
    left_tokens = tokenizer.tokenize(full[:idx_cands_start])
    cand1_tokens = tokenizer.tokenize(cand1)
    cand1_masks = [tokenizer.mask_token] * len(cand1_tokens)
    cand2_tokens = tokenizer.tokenize(cand2)
    cand2_masks = [tokenizer.mask_token] * len(cand2_tokens)
    mid_tokens = tokenizer.tokenize('than')
    right_tokens = tokenizer.tokenize(full[idx_cands_end:])
    
    all_tokens = [tokenizer.cls_token] + left_tokens + cand1_masks + mid_tokens + cand2_masks + right_tokens + [tokenizer.sep_token]

    # Create the input ids [input ids for left] [mask ids for cand1] [input ids for mid] [mask ids for cand2] [input ids for right]
    all_tokens = all_tokens[:MAX_SEQ_LEN]
    input_tokens = all_tokens + [tokenizer.pad_token] * (MAX_SEQ_LEN - len(all_tokens))
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)

    # Create masked lm labels  Create the masked_lm_labels [-100 for left tokens] [input ids for cand1] [-100 for mid tokens] [input ids for cand2] [-100 for right tokens]
    masked_lm_labels = [-100] + [-100] * len(left_tokens) + tokenizer.convert_tokens_to_ids(cand1_tokens) + \
                       [-100] * len(mid_tokens) + tokenizer.convert_tokens_to_ids(cand2_tokens) + [-100] * len(right_tokens) + [-100]

    
    masked_lm_labels = masked_lm_labels[:MAX_SEQ_LEN]
    masked_lm_labels += [-100] *  (MAX_SEQ_LEN - len(masked_lm_labels))
   
    # Create attention mask
    attention_mask = [1] * len(all_tokens)
    attention_mask += [0] *  (MAX_SEQ_LEN - len(attention_mask))
    
    # Create token type ids
    token_type_ids = [0] * MAX_SEQ_LEN
    
    # assertions
    #assert len(input_ids) == len(masked_lm_labels) and len(input_ids) == len(token_type_ids) and len(input_ids) == len(attention_mask)
    
    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    masked_lm_labels = torch.tensor([masked_lm_labels]).to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask, masked_lm_labels=masked_lm_labels)
        
    return outputs[0].to('cpu').item()


def bert_cross_entropy_for_winologic_s2(wl_sentence: WinoLogicSentence, model: BertForMaskedLM, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len
    
    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')
    
    # Create the tokens for the WinoLogic sentence with cand1 replaced by masks [tokens for left] [<mask>]*len(tokens for cand1) [tokens for right]
    left_tokens = tokenizer.tokenize(full[:idx_cands_start])
    cand1_tokens = tokenizer.tokenize(cand1)
    cand1_masks = [tokenizer.mask_token] * len(cand1_tokens)
    cand2_tokens = tokenizer.tokenize(cand2)
    cand2_masks = [tokenizer.mask_token] * len(cand2_tokens)
    mid_tokens = tokenizer.tokenize('than')
    right_tokens = tokenizer.tokenize(full[idx_cands_end:])
    
    all_tokens = [tokenizer.cls_token] + left_tokens + cand2_masks + mid_tokens + cand1_masks + right_tokens + [tokenizer.sep_token]

    
    # Create the input ids [input ids for left] [mask ids for cand1] [input ids for mid] [mask ids for cand2] [input ids for right]
    all_tokens = all_tokens[:MAX_SEQ_LEN]
    input_tokens = all_tokens + [tokenizer.pad_token] * (MAX_SEQ_LEN - len(all_tokens))
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    
    # Create masked lm labels  Create the masked_lm_labels [-100 for left tokens] [input ids for cand1] [-100 for mid tokens] [input ids for cand2] [-100 for right tokens]
    masked_lm_labels = [-100] + [-100] * len(left_tokens) + tokenizer.convert_tokens_to_ids(cand2_tokens) + \
                       [-100] * len(mid_tokens) + tokenizer.convert_tokens_to_ids(cand1_tokens) + [-100] * len(right_tokens) + [-100]
    masked_lm_labels = masked_lm_labels[:MAX_SEQ_LEN]
    masked_lm_labels += [-100] *  (MAX_SEQ_LEN - len(masked_lm_labels))
    
    # Create attention mask
    attention_mask = [1] * len(all_tokens)
    attention_mask += [0] *  (MAX_SEQ_LEN - len(attention_mask))
    
    # assertions
    #assert len(input_ids) == len(masked_lm_labels) and len(input_ids) == len(token_type_ids) and len(input_ids) == len(attention_mask)
    
    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    masked_lm_labels = torch.tensor([masked_lm_labels]).to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask, masked_lm_labels=masked_lm_labels)
        
    return outputs[0].to('cpu').item()


def bert_for_winologic(config: ExpConfig, bert_cross_entropy_for_s1, bert_cross_entropy_for_s2):
    """
    Using the BERT model (`config.model_name`) to solve WinoLogic,
    using `bert_cross_entropy_for_winologic` to compute the cross-entropy.
    Returns the list of accuracies.
    """
    # Load the WinoLogic dataset
    winologic_sentences = load_winologic_from_path(config.dataset_path)
        
    # Init the model, tokenizer
    if config.model_path is not None and config.model_path != "":
        model = BertForMaskedLM.from_pretrained(config.model_path)
        tokenizer = BertTokenizer.from_pretrained(config.model_path)
    else:
        tokenizer = BertTokenizer.from_pretrained(config.model_name)
        model = BertForMaskedLM.from_pretrained(config.model_name)
    model.eval()
    model.to(config.device)
    
    # calculuate the cross-entropy loss for each WinoLogic sentence
    #result = open(SAVE_RESULT_PATH, "a")
    #writer = csv.writer(result)
    #writer.writerow(["kID","correct", "valid", "sentence", "wsc_id", "false_type", "label", "predict"])
    
    correct = 0
    for ws in winologic_sentences:
        s1_loss = bert_cross_entropy_for_s1(ws, model, tokenizer, config.max_seq_len, config.device)
        s2_loss = bert_cross_entropy_for_s2(ws, model, tokenizer, config.max_seq_len, config.device)

        if (ws.label == 1 and s1_loss < s2_loss) or (ws.label == 0 and s1_loss > s2_loss):
            correct += 1
            #writer.writerow([ws.kID, 1, ws.valid, ws.sentence, ws.wsc_id, ws.false_type, ws.label, ws.label])
        #else:
            #writer.writerow([ws.kID, 0, ws.valid, ws.sentence, ws.wsc_id, ws.false_type, ws.label, 1 - ws.label])
        
    acc = correct / len(winologic_sentences)
    
    return acc

In [ ]:
for model_name in ['bert-base-uncased', 'bert-large-uncased', 'bert-base-uncased-wscr', 'bert-large-uncased-wscr', 'bert-base-uncased-winogrande', 'bert-large-uncased-winogrande']:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    bertconfig = ExpConfig()
    bertconfig.set_seed()
    bertconfig.set_gpu_if_possible(1)
    bertconfig.dataset = 'winologic'
    bertconfig.dataset_path = ROOT_PATH + WINOLOGIC_PATH
    bertconfig.task_name = 'Test on WinoLogic'
    bertconfig.model_name = model_name
    bertconfig.model_path = CACHE_DIR + model_name
    
    print('\n================================')
    print('Experiment: {} using {}'.format(bertconfig.task_name, bertconfig.model_name))
    
    acc = bert_for_winologic(bertconfig, bert_cross_entropy_for_winologic_s1, bert_cross_entropy_for_winologic_s2)
    
    print('Acc: {}'.format(acc))
    print('================================')

# Use RoBERTa

In [ ]:
def roberta_cross_entropy_for_winologic_s1(wl_sentence: WinoLogicSentence, model: BertForMaskedLM, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len
    
    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')
    
    # Create the tokens for the Wiq noLogic sentence with cand1 replaced by masks [tokens for left] [<mask>]*len(tokens for cand1) [tokens for mid] [<mask>]*len(tokens for cand2) [tokens for right]
    # before converting tokens into input_ids
    # The masked lm labels should be like [-100 for left] [input ids for cand1] [-100 for mid] [input ids for cand2] [-100 for right]
    # The attention mask is used to avoid attention for the padding tokens
    # All the inputs are padded accordingly to the length of MAX_SEQ_LEN: zero-pad for input_ids and attention_mask, -100-pad for masked_lm_labels
    # assert len(tokenizer.tokenize(full, add_prefix_space=True)) == len(masked_sentence_tokens)
    left_tokens = tokenizer.tokenize(full[:idx_cands_start], add_prefix_space=True)
    cand1_tokens = tokenizer.tokenize(cand1, add_prefix_space=True)
    cand1_masks = [tokenizer.mask_token] * len(cand1_tokens)
    cand2_tokens = tokenizer.tokenize(cand2, add_prefix_space=True)
    cand2_masks = [tokenizer.mask_token] * len(cand2_tokens)
    mid_tokens = tokenizer.tokenize('than', add_prefix_space=True)
    right_tokens = tokenizer.tokenize(full[idx_cands_end:], add_prefix_space=True)
    #assert len(left_tokens + cand1_tokens + mid_tokens + cand2_tokens + right_tokens) == len(tokenizer.tokenize(full, add_prefix_space=True)) 
    # Convert masked sentence into input_ids
    input_ids = tokenizer.convert_tokens_to_ids(left_tokens + cand1_masks + mid_tokens + cand2_masks + right_tokens)
    # Create the masked_lm_labels [-100 for left tokens] [input ids for cand1] [-100 for mid tokens] [input ids for cand2] [-100 for right tokens]
    masked_lm_labels = [-100]*len(left_tokens) + \
                       tokenizer.convert_tokens_to_ids(cand1_tokens) + \
                       [-100]*len(mid_tokens) +  \
                       tokenizer.convert_tokens_to_ids(cand2_tokens) + \
                       [-100]*len(right_tokens)
    #assert len(mid_tokens) == 1
    # Truncate the input_ids and masked_lm_labels if needed
    input_ids = input_ids[:MAX_SEQ_LEN]
    masked_lm_labels = masked_lm_labels[:MAX_SEQ_LEN]
    
    # Create the attention_mask
    attention_mask = [1] * len(input_ids)
    
    # Zero-pad the input_ids and attention_mask, -100-pad the masked_lm_labels
    input_ids += [0] * (MAX_SEQ_LEN - len(input_ids))
    attention_mask += [0] * (MAX_SEQ_LEN - len(attention_mask))
    masked_lm_labels += [-100] * (MAX_SEQ_LEN - len(masked_lm_labels))
    
    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    masked_lm_labels = torch.tensor([masked_lm_labels]).to(device)

    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask, masked_lm_labels=masked_lm_labels)
        
    return outputs[0].to('cpu').item()


def roberta_cross_entropy_for_winologic_s2(wl_sentence: WinoLogicSentence, model: BertForMaskedLM, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len
    
    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')
    
    # Create the tokens for the WinoLogic sentence with cand1 replaced by masks [tokens for left] [<mask>]*len(tokens for cand1) [tokens for mid] [<mask>]*len(tokens for cand2) [tokens for right]
    # before converting tokens into input_ids
    # The masked lm labels should be like [-100 for left] [input ids for cand1] [-100 for mid] [input ids for cand2] [-100 for right]
    # The attention mask is used to avoid attention for the padding tokens
    # All the inputs are padded accordingly to the length of MAX_SEQ_LEN: zero-pad for input_ids and attention_mask, -100-pad for masked_lm_labels
    # assert len(tokenizer.tokenize(full, add_prefix_space=True)) == len(masked_sentence_tokens)
    left_tokens = tokenizer.tokenize(full[:idx_cands_start], add_prefix_space=True)
    cand1_tokens = tokenizer.tokenize(cand1, add_prefix_space=True)
    cand1_masks = [tokenizer.mask_token] * len(cand1_tokens)
    cand2_tokens = tokenizer.tokenize(cand2, add_prefix_space=True)
    cand2_masks = [tokenizer.mask_token] * len(cand2_tokens)
    mid_tokens = tokenizer.tokenize('than', add_prefix_space=True)
    right_tokens = tokenizer.tokenize(full[idx_cands_end:], add_prefix_space=True)
    #assert len(left_tokens + cand1_tokens + mid_tokens + cand2_tokens + right_tokens) == len(tokenizer.tokenize(full, add_prefix_space=True)) 
    # Convert masked sentence into input_ids
    input_ids = tokenizer.convert_tokens_to_ids(left_tokens + cand2_masks + mid_tokens + cand1_masks + right_tokens)
    # Create the masked_lm_labels [-100 for left tokens] [input ids for cand1] [-100 for mid tokens] [input ids for cand2] [-100 for right tokens]
    masked_lm_labels = [-100]*len(left_tokens) + \
                       tokenizer.convert_tokens_to_ids(cand2_tokens) + \
                       [-100]*len(mid_tokens) +  \
                       tokenizer.convert_tokens_to_ids(cand1_tokens) + \
                       [-100]*len(right_tokens)
    #assert len(mid_tokens) == 1
    # Truncate the input_ids and masked_lm_labels if needed
    input_ids = input_ids[:MAX_SEQ_LEN]
    masked_lm_labels = masked_lm_labels[:MAX_SEQ_LEN]
    
    # Create the attention_mask
    attention_mask = [1] * len(input_ids)
    
    # Zero-pad the input_ids and attention_mask, -100-pad the masked_lm_labels
    input_ids += [0] * (MAX_SEQ_LEN - len(input_ids))
    attention_mask += [0] * (MAX_SEQ_LEN - len(attention_mask))
    masked_lm_labels += [-100] * (MAX_SEQ_LEN - len(masked_lm_labels))
    
    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    masked_lm_labels = torch.tensor([masked_lm_labels]).to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask, masked_lm_labels=masked_lm_labels)
        
    return outputs[0].to('cpu').item()

def roberta_for_winologic(config: ExpConfig, roberta_cross_entropy_for_winologic_s1, roberta_cross_entropy_for_winologic_s2):
    """
    Using the RoBERTa model (`config.model_name`) to solve WinoLogic,
    using `roberta_cross_entropy_for_winologic` to compute the cross-entropy.
    Returns the list of accuracies.
    """
    # Load the WinoLogic dataset
    winologic_sentences = load_winologic_from_path(config.dataset_path)
        
    # Init the model, tokenizer
    if config.model_path is not None and config.model_path != "":
        tokenizer = RobertaTokenizer.from_pretrained(config.model_path)
        model = RobertaForMaskedLM.from_pretrained(config.model_path)
    else:
        tokenizer = RobertaTokenizer.from_pretrained(model_name)
        model = RobertaForMaskedLM.from_pretrained(model_name)
    model.eval()
    model.to(config.device)
    
    #result = open(SAVE_RESULT_PATH, "a")
    #writer = csv.writer(result)
    #writer.writerow(["kID","correct", "valid", "sentence", "wsc_id", "false_type", "label", "predict"])
    # calculuate the cross-entropy loss for each WinoLogic sentence
    correct = 0
    for ws in winologic_sentences:
        s1_loss = roberta_cross_entropy_for_winologic_s1(ws, model, tokenizer, config.max_seq_len, config.device)
        s2_loss = roberta_cross_entropy_for_winologic_s2(ws, model, tokenizer, config.max_seq_len, config.device)
        
        if (ws.label == 1 and s1_loss < s2_loss) or (ws.label == 0 and s1_loss > s2_loss):
            correct += 1
            #writer.writerow([ws.kID, 1, ws.valid, ws.sentence, ws.wsc_id, ws.false_type, ws.label, ws.label])
        #else:
            #writer.writerow([ws.kID, 0, ws.valid, ws.sentence, ws.wsc_id, ws.false_type, ws.label, 1 - ws.label])
            
    acc = correct / len(winologic_sentences)
    
    return acc

In [ ]:
for model_name in ['roberta-base', 'roberta-large', 'roberta-base-wscr', 'roberta-large-wscr', 'roberta-base-winogrande', 'roberta-large-winogrande']:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    robertaconfig = ExpConfig()
    robertaconfig.set_seed()
    robertaconfig.set_gpu_if_possible(1)
    robertaconfig.dataset = 'winologic'
    robertaconfig.dataset_path = ROOT_PATH + WINOLOGIC_PATH
    robertaconfig.task_name = 'Test on WinoLogic'
    robertaconfig.model_name = model_name
    robertaconfig.model_path = CACHE_DIR + model_name
    
    print('\n================================')
    print('Experiment: {} using {}'.format(robertaconfig.task_name, robertaconfig.model_name))
    
    scores = roberta_for_winologic(robertaconfig, roberta_cross_entropy_for_winologic_s1, roberta_cross_entropy_for_winologic_s2)
    
    print('Scores: {}'.format(scores))
    print('================================')

## Use BERT (QNLI)

In [4]:
def bert_cross_entropy_for_winologic_nli(wl_sentence: WinoLogicSentence, model: BertForMaskedLM, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len
    
    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')    
    
    wsc_sentence_candidate = wl_sentence.wsc_marked_sentence.split('*')[0] + cand1 + wl_sentence.wsc_marked_sentence.split('*')[2]
    
    first = tokenizer.tokenize(wl_sentence.knowledge)
    second = tokenizer.tokenize(wsc_sentence_candidate)

    all_tokens = [tokenizer.cls_token] + first + [tokenizer.sep_token] + second + [tokenizer.sep_token]

    
    # Create the input ids [input ids for left] [mask ids for cand1] [input ids for mid] [mask ids for cand2] [input ids for right]
    all_tokens = all_tokens[:MAX_SEQ_LEN]
    input_tokens = all_tokens + [tokenizer.pad_token] * (MAX_SEQ_LEN - len(all_tokens))
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    
    # Create attention mask
    attention_mask = [1] * len(all_tokens)
    attention_mask += [0] *  (MAX_SEQ_LEN - len(attention_mask))
    
    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask)
        
    return torch.argmax(outputs[0]).to('cpu').item()


def bert_for_winologic(config: ExpConfig, bert_cross_entropy_for_winologic_nli):
    """
    Using the BERT model (`config.model_name`) to solve WinoLogic,
    using `bert_cross_entropy_for_winologic` to compute the cross-entropy.
    Returns the list of accuracies.
    """
    # Load the WinoLogic dataset
    winologic_sentences = load_winologic_from_path(config.dataset_path)
        
    # Init the model, tokenizer
    if config.model_path is not None and config.model_path != "":
        model = BertForSequenceClassification.from_pretrained(config.model_path)
        tokenizer = BertTokenizer.from_pretrained(config.model_path)
    else:
        tokenizer = BertTokenizer.from_pretrained(config.model_name)
        model = BertForSequenceClassification.from_pretrained(config.model_name)
    model.eval()
    model.to(config.device)
    
    # calculuate the cross-entropy loss for each WinoLogic sentence
    correct = 0
    #result = open(SAVE_RESULT_PATH, "a")
    #writer = csv.writer(result)
    #writer.writerow(["kID", "pred", "label", "correct", "valid", "sentence", "wsc_id", "false_type"])
    
    for ws in winologic_sentences:
        pred = bert_cross_entropy_for_winologic_nli(ws, model, tokenizer, config.max_seq_len, config.device)
        if pred == 1:
            pred_str = "entailment"
        elif pred == 0:
            pred_str = "not entailment"
        if pred == ws.label:
            correct += 1
            #writer.writerow([ws.kID, pred_str, ws.label, 1, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
        #else:
            #writer.writerow([ws.kID, pred_str, ws.label, 0, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
    acc = correct / len(winologic_sentences)
    return acc

In [ ]:
for model_name in ['bert-base-uncased-qnli']:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    bertconfig = ExpConfig()
    bertconfig.set_seed()
    bertconfig.set_gpu_if_possible(1)
    bertconfig.dataset = 'winologic'
    bertconfig.dataset_path = ROOT_PATH + WINOLOGIC_PATH
    bertconfig.task_name = 'Test on WinoLogic'
    bertconfig.model_name = model_name
    bertconfig.model_path = CACHE_DIR + model_name
    
    print('\n================================')
    print('Experiment: {} using {}'.format(bertconfig.task_name, bertconfig.model_name))
    
    acc = bert_for_winologic(bertconfig, bert_cross_entropy_for_winologic_nli)
    
    print('Acc: {}'.format(acc))
    print('================================')

## Use BERT (MNLI)

In [ ]:
def bert_cross_entropy_for_winologic_nli(wl_sentence: WinoLogicSentence, model: BertForMaskedLM, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len

    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')    
    
    wsc_sentence_candidate = wl_sentence.wsc_marked_sentence.split('*')[0] + cand1 + wl_sentence.wsc_marked_sentence.split('*')[2]
    
    first = tokenizer.tokenize(wl_sentence.knowledge)
    second = tokenizer.tokenize(wsc_sentence_candidate)
    
    all_tokens = [tokenizer.cls_token] + first + [tokenizer.sep_token] + second + [tokenizer.sep_token]

    # Create the input ids [input ids for left] [mask ids for cand1] [input ids for mid] [mask ids for cand2] [input ids for right]
    all_tokens = all_tokens[:MAX_SEQ_LEN]
    input_tokens = all_tokens + [tokenizer.pad_token] * (MAX_SEQ_LEN - len(all_tokens))
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    
    # Create attention mask
    attention_mask = [1] * len(all_tokens)
    attention_mask += [0] *  (MAX_SEQ_LEN - len(attention_mask))

    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask)
        
    return torch.argmax(outputs[0]).to('cpu').item()


def bert_for_winologic(config: ExpConfig, bert_cross_entropy_for_winologic_nli):
    """
    Using the BERT model (`config.model_name`) to solve WinoLogic,
    using `bert_cross_entropy_for_winologic` to compute the cross-entropy.
    Returns the list of accuracies.
    """
    # Load the WinoLogic dataset
    winologic_sentences = load_winologic_from_path(config.dataset_path)
        
    # Init the model, tokenizer
    if config.model_path is not None and config.model_path != "":
        model = BertForSequenceClassification.from_pretrained(config.model_path)
        tokenizer = BertTokenizer.from_pretrained(config.model_path)
    else:
        tokenizer = BertTokenizer.from_pretrained(config.model_name)
        model = BertForSequenceClassification.from_pretrained(config.model_name)
    model.eval()
    model.to(config.device)
    
    # calculuate the cross-entropy loss for each WinoLogic sentence
    correct = 0
    #result = open(SAVE_RESULT_PATH, "a")
    #writer = csv.writer(result)
    #writer.writerow(["kID", "pred", "label", "correct", "valid", "sentence", "wsc_id", "false_type"])
    for ws in winologic_sentences:
        pred = bert_cross_entropy_for_winologic_nli(ws, model, tokenizer, config.max_seq_len, config.device)
        if pred == 2:
            pred_str = "entailment"
        elif pred == 0:
            pred_str = "contradiction"
        elif pred == 1:
            pred_str = "neutral"
        if (pred == 0 and ws.label == 0) or (pred == 1 and ws.label == 0) or (pred == 2 and ws.label == 1):
            correct += 1
            #writer.writerow([ws.kID, pred_str, ws.label, 1, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
        #else:
            #writer.writerow([ws.kID, pred_str, ws.label, 0, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
    acc = correct / len(winologic_sentences)
    return acc

In [ ]:
for model_name in ['bert-base-uncased-mnli']:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    bertconfig = ExpConfig()
    bertconfig.set_seed()
    bertconfig.set_gpu_if_possible(1)
    bertconfig.dataset = 'winologic'
    bertconfig.dataset_path = ROOT_PATH + WINOLOGIC_PATH
    bertconfig.task_name = 'Test on WinoLogic'
    bertconfig.model_name = model_name
    bertconfig.model_path = CACHE_DIR + model_name
    
    print('\n================================')
    print('Experiment: {} using {}'.format(bertconfig.task_name, bertconfig.model_name))
    
    acc = bert_for_winologic(bertconfig, bert_cross_entropy_for_winologic_nli)
    
    print('Acc: {}'.format(acc))
    print('================================')

## Use RoBERTa (QNLI)

In [ ]:
def roberta_cross_entropy_for_winologic_nli(wl_sentence: WinoLogicSentence, model: RobertaForSequenceClassification, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len

    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')    
    
    wsc_sentence_candidate = wl_sentence.wsc_marked_sentence.split('*')[0] + cand1 + wl_sentence.wsc_marked_sentence.split('*')[2]
    
    first = tokenizer.tokenize(wl_sentence.knowledge, add_prefix_space=True)
    second = tokenizer.tokenize(wsc_sentence_candidate, add_prefix_space=True)

    input_ids = tokenizer.convert_tokens_to_ids(first + [tokenizer.sep_token] + second)

    #assert len(mid_tokens) == 1
    # Truncate the input_ids and masked_lm_labels if needed
    input_ids = input_ids[:MAX_SEQ_LEN]
    
    # Create the attention_mask
    attention_mask = [1] * len(input_ids)
    
    # Zero-pad the input_ids and attention_mask, -100-pad the masked_lm_labels
    input_ids += [0] * (MAX_SEQ_LEN - len(input_ids))
    attention_mask += [0] * (MAX_SEQ_LEN - len(attention_mask))
    
    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask)
        
    return torch.argmax(outputs[0]).to('cpu').item()

def roberta_for_winologic(config: ExpConfig, roberta_cross_entropy_for_winologic_nli):
    """
    Using the RoBERTa model (`config.model_name`) to solve WinoLogic,
    using `roberta_cross_entropy_for_winologic` to compute the cross-entropy.
    Returns the list of accuracies.
    """
    # Load the WinoLogic dataset
    winologic_sentences = load_winologic_from_path(config.dataset_path)
        
    # Init the model, tokenizer
    if config.model_path is not None and config.model_path != "":
        tokenizer = RobertaTokenizer.from_pretrained(config.model_path)
        model = RobertaForSequenceClassification.from_pretrained(config.model_path)
    else:
        tokenizer = RobertaTokenizer.from_pretrained(model_name)
        model = RobertaForSequenceClassification.from_pretrained(model_name)
    model.eval()
    model.to(config.device)
    
    # calculuate the cross-entropy loss for each WinoLogic sentence
    correct = 0
    #result = open(SAVE_RESULT_PATH, "a")
    #writer = csv.writer(result)
    #writer.writerow(["kID", "pred", "label", "correct", "valid", "sentence", "wsc_id", "false_type"])
    for ws in winologic_sentences:
        pred = roberta_cross_entropy_for_winologic_nli(ws, model, tokenizer, config.max_seq_len, config.device)
        if pred == 0:
            pred_str = "entailment"
        elif pred == 1:
            pred_str = "not entailment"
        if pred != ws.label:
            correct += 1
        #    writer.writerow([ws.kID, pred_str, ws.label, 1, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
        #else:
        #    writer.writerow([ws.kID, pred_str, ws.label, 0, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
    acc = correct / len(winologic_sentences)

    return acc

In [ ]:
for model_name in ['roberta-base-qnli']:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    robertaconfig = ExpConfig()
    robertaconfig.set_seed()
    robertaconfig.set_gpu_if_possible(1)
    robertaconfig.dataset = 'winologic'
    robertaconfig.dataset_path = ROOT_PATH + WINOLOGIC_PATH
    robertaconfig.task_name = 'Test on WinoLogic'
    robertaconfig.model_name = model_name
    robertaconfig.model_path = CACHE_DIR + model_name
    
    print('\n================================')
    print('Experiment: {} using {}'.format(robertaconfig.task_name, robertaconfig.model_name))
    
    scores = roberta_for_winologic(robertaconfig, roberta_cross_entropy_for_winologic_nli)
    
    print('Scores: {}'.format(scores))
    print('================================')

## Use Roberta (MNLI)

In [ ]:
def roberta_cross_entropy_for_winologic_nli(wl_sentence: WinoLogicSentence, model: RobertaForSequenceClassification, tokenizer: BertTokenizer, max_seq_len: int, device: str='cpu') -> float:
    """
    Predict both the first and second candidates in WinoLogic sentence, and return the cross entropy loss
    """
    MAX_SEQ_LEN = max_seq_len

    full = wl_sentence.sentence
    # the pattern to find <cand1> and <cand2>: ... is more likely to refer to <cand1> than <cand2> because ...
    pat_cand = r'(?<=is more likely to refer to ).*?(?= than ).*?(?= because )'
    # Extract <cand1> and <cand2> along with their positions
    m = re.search(pat_cand, full)
    idx_cands_start = m.start()
    idx_cands_end = m.end()
    cand1, cand2 = m.group().split(' than ')    
    
    wsc_sentence_candidate = wl_sentence.wsc_marked_sentence.split('*')[0] + cand1 + wl_sentence.wsc_marked_sentence.split('*')[2]
    
    first = tokenizer.tokenize(wl_sentence.knowledge, add_prefix_space=True)
    second = tokenizer.tokenize(wsc_sentence_candidate, add_prefix_space=True)

    input_ids = tokenizer.convert_tokens_to_ids(first + [tokenizer.sep_token] + second)

    # Truncate the input_ids and masked_lm_labels if needed
    input_ids = input_ids[:MAX_SEQ_LEN]
    
    # Create the attention_mask
    attention_mask = [1] * len(input_ids)
    
    # Zero-pad the input_ids and attention_mask, -100-pad the masked_lm_labels
    input_ids += [0] * (MAX_SEQ_LEN - len(input_ids))
    attention_mask += [0] * (MAX_SEQ_LEN - len(attention_mask))
    
    input_ids = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([attention_mask]).to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask)
        
    return torch.argmax(outputs[0]).to('cpu').item()

def roberta_for_winologic(config: ExpConfig, roberta_cross_entropy_for_winologic_nli):
    """
    Using the RoBERTa model (`config.model_name`) to solve WinoLogic,
    using `roberta_cross_entropy_for_winologic` to compute the cross-entropy.
    Returns the list of accuracies.
    """
    # Load the WinoLogic dataset
    winologic_sentences = load_winologic_from_path(config.dataset_path)
        
    # Init the model, tokenizer
    if config.model_path is not None and config.model_path != "":
        tokenizer = RobertaTokenizer.from_pretrained(config.model_path)
        model = RobertaForSequenceClassification.from_pretrained(config.model_path)
    else:
        tokenizer = RobertaTokenizer.from_pretrained(model_name)
        model = RobertaForSequenceClassification.from_pretrained(model_name)
    model.eval()
    model.to(config.device)
    
    # calculuate the cross-entropy loss for each WinoLogic sentence
    correct = 0
    #result = open(SAVE_RESULT_PATH, "a")
    #writer = csv.writer(result)
    #writer.writerow(["kID", "pred", "label", "correct", "valid", "sentence", "wsc_id", "false_type"])
    for ws in winologic_sentences:
        pred = roberta_cross_entropy_for_winologic_nli(ws, model, tokenizer, config.max_seq_len, config.device)
        if pred == 2:
            pred_str = "entailment"
        elif pred == 0:
            pred_str = "contradiction"
        elif pred == 1:
            pred_str = "neutral"
        if (pred == 0 and ws.label == 0) or (pred == 1 and ws.label == 0) or (pred == 2 and ws.label == 1):
            correct += 1
        #    writer.writerow([ws.kID, pred_str, ws.label, 1, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
        #else:
        #    writer.writerow([ws.kID, pred_str, ws.label, 0, ws.valid, ws.sentence, ws.wsc_id, ws.false_type])
    acc = correct / len(winologic_sentences)

    return acc

In [ ]:
for model_name in ['roberta-base-mnli', 'roberta-large-mnli']:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    robertaconfig = ExpConfig()
    robertaconfig.set_seed()
    robertaconfig.set_gpu_if_possible(1)
    robertaconfig.dataset = 'winologic'
    robertaconfig.dataset_path = ROOT_PATH + WINOLOGIC_PATH
    robertaconfig.task_name = 'Test on WinoLogic, predicting nli'
    robertaconfig.model_name = model_name
    robertaconfig.model_path = CACHE_DIR + model_name
    
    print('\n================================')
    print('Experiment: {} using {}'.format(robertaconfig.task_name, robertaconfig.model_name))
    
    scores = roberta_for_winologic(robertaconfig, roberta_cross_entropy_for_winologic_nli)
    
    print('Scores: {}'.format(scores))
    print('================================')